In [40]:
import pandas as pd
import random
import numpy as np
import time
import math
import sys
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import datatable as dt

from xgboost import plot_tree
from os import walk
from os import listdir
from os.path import isfile, join
from scipy.spatial.distance import euclidean, pdist, squareform
from scipy.stats import skew
from scipy.special import expit as sigmoid
from scipy.cluster.hierarchy import fclusterdata
from pandas.plotting import autocorrelation_plot
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import precision_recall_fscore_support
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import pairwise_distances
from sklearn.cluster import AgglomerativeClustering
from sklearn.manifold import TSNE
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [41]:
ny_cols = ["last_name", "first_name", "middle_name", "name_suffix", "house_number", "house_fractional_addr", "residence_apartment", "residence_pre_street_direction", "residence_street_name", "residence_post_street_direction", "residence_city", "residence_zip_code_5", "residence_zip_code_4", "mail_addr1", "mail_addr2", "mail_addr3", "mail_addr4", "dob", "gender", "political_party", "other_party", "county_code", "election_district", "legislative_district", "town_city", "ward", "congressional_district", "senate_district", "assembly_district", "last_date_voted", "last_year_voted", "last_county_voted", "last_registered_address", "last_registered_name", "county_voter_registration_no", "application_date", "application_source", "identification_required_flag", "identification_verification_requirement_met_flag", "voter_status_codes", "status_reason_codes", "inactive_voter_date", "purge_voter_date", "unique_nys_voter_id", "voter_history"]


In [42]:
selective_headers = [
    'first_name',
    'last_name',
    'dob',
    'county_code',
    'house_number',
    'residence_apartment',
    'residence_street_name',
    'residence_city',
    'residence_zip_code_5',
    'gender',
    'unique_nys_voter_id',
    'political_party',
    'voter_status_codes'
]

# precinct and race was not found

In [43]:
new_york_path = "data/NewYork"

In [44]:
COUNTY=sys.argv[1]

In [45]:
date_str = sys.argv[2]

## Uncomment below lines and add parameters manually when exploring through notebook instead of python-script

In [46]:
date_str = "20160511"
COUNTY = "31"

In [47]:
import os
if not os.path.exists(new_york_path + "/couples/" + date_str):
    os.makedirs(new_york_path + "/couples/" + date_str)

In [48]:
source_county_file_name = "county_" + date_str + "_" + COUNTY + ".csv"

In [49]:
source_county_file_name

'county_20160511_31.csv'

In [50]:
COUPLES_SAVED_PATH = new_york_path + "/couples/" + date_str + "/" + "couples_" + date_str + "_" + COUNTY + ".csv"

In [13]:
COUPLES_SAVED_PATH

'data/NewYork/couples/20121231/couples_20121231_31.csv'

In [51]:
new_york_path + "/" + date_str + "_county_files/" + source_county_file_name

'data/NewYork/20160511_county_files/county_20160511_31.csv'

In [52]:
global_df = pd.read_csv(new_york_path + "/" + date_str + "_county_files/" + source_county_file_name, sep="\t",  encoding='iso-8859-1')

In [53]:
global_df.head()

,last_name,first_name,house_number,residence_apartment,residence_street_name,residence_city,residence_zip_code_5,dob,gender,political_party,county_code,voter_status_codes,unique_nys_voter_id,age,uniq_addr
0,barrera,john,237,1,west 11 street,manhattan,10014.0,19720228,M,DEM,31,ACTIVE,NY000000000034125923,44,237 1 west 11 street manhattan 100140
1,aranda,daisy,5,1814,tudor city place,new york,10017.0,19581027,F,DEM,31,PURGED,NY000000000034135787,57,5 1814 tudor city place new york 100170
2,reilly,betty,800,6c,riverside drive,new york,10032.0,19220513,F,DEM,31,PURGED,NY000000000034153789,93,800 6c riverside drive new york 100320
3,zilberman,pavel,354,apt 2104,east 91 street,new york,10128.0,19531117,M,REP,31,ACTIVE,NY000000000034164964,62,354 apt 2104 east 91 street new york 101280
4,thomas,adrian,165,5,east 87 street,new york,10128.0,19720413,M,IND,31,ACTIVE,NY000000000034166816,44,165 5 east 87 street new york 101280


In [54]:
global_df.shape

(1428253, 15)

In [55]:
global_df_copy = global_df.copy(deep=True)

In [56]:
merge = pd.merge(global_df, global_df_copy, on=["uniq_addr"], suffixes=["_L", "_R"])

In [57]:
merge = merge[merge["unique_nys_voter_id_L"] != merge["unique_nys_voter_id_R"]]

In [58]:
merge.shape

(1118414, 29)

In [59]:
filtered = merge[merge["unique_nys_voter_id_L"] < merge["unique_nys_voter_id_R"]]

In [60]:
filtered.shape

(559207, 29)

In [61]:
def modified_couple_heuristic(row):
    male_age_threshold = 27
    female_age_threshold = 25
    unknown_age_threshold = 26
    age_diff_threshold = 15
    
    age_diff = abs(row['age_L'] - row['age_R'])
    
    is_age_threshold_L = False
    if row["gender_L"] == "M" and row["age_L"] >= male_age_threshold: 
        is_age_threshold_L = True
    elif row["gender_L"] == "F" and row["age_L"] >= female_age_threshold:
        is_age_threshold_L = True
    elif row["gender_L"] == "U" and row["age_L"] >= unknown_age_threshold:
        is_age_threshold_L = True

    
        
    is_age_threshold_R = False
    if row["gender_R"] == "M" and row["age_R"] >= male_age_threshold: 
        is_age_threshold_R = True
    elif row["gender_R"] == "F" and row["age_R"] >= female_age_threshold:
        is_age_threshold_R = True
    elif row["gender_R"] == "U" and row["age_R"] >= unknown_age_threshold:
        is_age_threshold_R = True
    
    return is_age_threshold_L and is_age_threshold_R and age_diff <= age_diff_threshold
        

In [62]:
filtered.columns

Index(['last_name_L', 'first_name_L', 'house_number_L',
       'residence_apartment_L', 'residence_street_name_L', 'residence_city_L',
       'residence_zip_code_5_L', 'dob_L', 'gender_L', 'political_party_L',
       'county_code_L', 'voter_status_codes_L', 'unique_nys_voter_id_L',
       'age_L', 'uniq_addr', 'last_name_R', 'first_name_R', 'house_number_R',
       'residence_apartment_R', 'residence_street_name_R', 'residence_city_R',
       'residence_zip_code_5_R', 'dob_R', 'gender_R', 'political_party_R',
       'county_code_R', 'voter_status_codes_R', 'unique_nys_voter_id_R',
       'age_R'],
      dtype='object')

In [63]:
filtered.shape

(559207, 29)

In [64]:
couples = filtered[filtered.apply(modified_couple_heuristic, axis=1)]

In [65]:
couples.shape

(302999, 29)

In [66]:
couples.head()

,last_name_L,first_name_L,house_number_L,residence_apartment_L,residence_street_name_L,residence_city_L,residence_zip_code_5_L,dob_L,gender_L,political_party_L,...,residence_street_name_R,residence_city_R,residence_zip_code_5_R,dob_R,gender_R,political_party_R,county_code_R,voter_status_codes_R,unique_nys_voter_id_R,age_R
14,carr,adam,453,19,west 36 street,manhattan,10018.0,19710916,M,DEM,...,west 36 street,manhattan,10018.0,19581007,M,GRE,31,ACTIVE,NY000000000037934705,57
18,serpico,joseph,15,apt 2220,broad street,new york,10005.0,19590423,M,REP,...,broad street,new york,10005.0,19670310,F,REP,31,ACTIVE,NY000000000034319286,49
22,richman,eric,240,b,east 90 street,manhattan,10128.0,19711231,M,IND,...,east 90 street,manhattan,10128.0,19731215,F,DEM,31,INACTIVE,NY000000000037320071,42
36,andrescavage,michael,15,7,abingdon square,manhattan,10014.0,19800403,M,REP,...,abingdon square,manhattan,10014.0,19810331,F,IND,31,ACTIVE,NY000000000051288300,35
45,zakai,howard,171,10d,east 84 street,manhattan,10028.0,19800318,M,DEM,...,east 84 street,manhattan,10028.0,19800721,F,DEM,31,PURGED,NY000000000051588364,35


In [67]:
global_df.shape

(1428253, 15)

In [68]:
couples["age_diff"] = couples.apply(lambda row: abs(row["age_L"] - row["age_R"]), axis=1)

/home/rahullachh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [69]:
sorted_couples = couples.sort_values(by="age_diff")

In [70]:
single_house_couples = sorted_couples.drop_duplicates(subset="uniq_addr", keep="first")

In [71]:
single_house_couples.shape

(197976, 30)

In [72]:
global_df.shape

(1428253, 15)

In [73]:
COUPLES_SAVED_PATH

'data/NewYork/couples/20160511/couples_20160511_31.csv'

In [74]:
single_house_couples.to_csv(COUPLES_SAVED_PATH, sep="\t", index=False)

In [75]:
print("Done processing " + COUPLES_SAVED_PATH)

Done processing data/NewYork/couples/20160511/couples_20160511_31.csv
